# Membuat Dataset untuk Model R-CNN

steps:
1. convert image ke array, ambil bagian tengah dari gambar saja (range dari y (148,308))
2. ambil nilai array dalam range boundingbox, array tersebut adalah object mobil
3. buat proposed box
4. bandingkan IoU antara boundingbox dengan proposed box, ambil nilai diatas 60% sebagai true, sisanya false
5. tulis array sebagai gambar JPG dengan folder class masing-masing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

import tensorflow as tf
import tensorflow_hub

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img

In [2]:
# https://learnopencv.com/intersection-over-union-iou-in-object-detection-and-segmentation/
def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])
    
    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))
    
    area_of_intersection = i_height * i_width
    
    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1
    
    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1
    
    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection
    
    iou = area_of_intersection / area_of_union
    
    return iou

In [3]:
img_w = 676
img_h = 380

target_w = 224
target_h = 224

In [4]:
df_boundingbox = pd.read_csv('../../data/train_solution_bounding_boxes.csv')
df_boundingbox['xmin'] = df_boundingbox['xmin'].astype('int')
df_boundingbox['ymin'] = df_boundingbox['ymin'].astype('int')
df_boundingbox['xmax'] = df_boundingbox['xmax'].astype('int')
df_boundingbox['ymax'] = df_boundingbox['ymax'].astype('int')
df_boundingbox['area'] = (df_boundingbox['xmax']-df_boundingbox['xmin']) * (df_boundingbox['ymax']-df_boundingbox['ymin'])

bb_data = {}

for index,row in df_boundingbox.iterrows():
    # filename, xmin, ymin, xmax, ymax = row['image'], int(row['xmin'] * target_w), int(row['ymin'] * target_h), int(row['xmax'] * target_w), int(row['ymax'] * target_h)
    filename, xmin, ymin, xmax, ymax = row['image'], row['xmin'], row['ymin'], row['xmax'], row['ymax']

    if not filename in bb_data:
        bb_data[filename] = [[xmin, ymin, xmax, ymax]]
    else:
        bb_data[filename].append([xmin, ymin, xmax, ymax])

print(len(bb_data))

355


In [5]:
df_boundingbox.describe()

,xmin,ymin,xmax,ymax,area
count,559.000000,559.000000,559.000000,559.000000,559.000000
mean,262.951699,185.980322,364.871199,231.007156,5067.200358
std,207.319450,9.297433,213.590834,14.001790,3994.887199
min,0.000000,148.000000,28.000000,198.000000,361.000000
25%,61.000000,180.000000,166.000000,223.000000,2658.000000
50%,236.000000,187.000000,348.000000,230.000000,4116.000000
75%,446.500000,193.000000,571.000000,237.000000,6040.000000
max,645.000000,208.000000,676.000000,308.000000,39468.000000


In [6]:
import random
import shutil

label1_files = df_boundingbox['image'].unique()
train_files = tf.io.gfile.glob('../../data/training_images/*.jpg')
train_files = [x.split('\\')[-1] for x in train_files]
random.shuffle(train_files)
label0_files = [x for x in train_files if x not in label1_files]

print(len(label1_files), len(label0_files), len(train_files))

val_size = 0.2
label0_val = int(len(label0_files) * val_size)
label1_val = int(len(label1_files) * val_size)

label0_train_files = label0_files[label0_val:]
label0_val_files = label0_files[:label0_val]
label1_train_files = label1_files[label1_val:]
label1_val_files = label1_files[:label1_val]
print(len(label0_train_files),len(label1_train_files),len(label0_val_files),len(label1_val_files))

for i in tf.io.gfile.glob('../../data/training_images/*.jpg'):
    filename = i.split('\\')[-1]
    
    if filename in label0_train_files:
        dst = '../../data/training_images_labeled-3/train/0/' + filename
        shutil.copyfile(i, dst)
    if filename in label1_train_files:
        dst = '../../data/training_images_labeled-3/train/1/' + filename
        shutil.copyfile(i, dst)
    if filename in label0_val_files:
        dst = '../../data/training_images_labeled-3/val/0/' + filename
        shutil.copyfile(i, dst)
    if filename in label1_val_files:
        dst = '../../data/training_images_labeled-3/val/1/' + filename
        shutil.copyfile(i, dst)

355 646 1001
517 284 129 71


In [7]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
used_image_size = (148,308)
max_proposed_boxes = 5000

train_files_0 = tf.io.gfile.glob('../../data/training_images_labeled-3/train/0/*.jpg')
train_files_1 = tf.io.gfile.glob('../../data/training_images_labeled-3/train/1/*.jpg')

train_files = train_files_0 + train_files_1

pos_counter = 0
neg_counter = 0

for i in train_files:
    filename = i.split('\\')[-1]
    
    if filename in df_boundingbox['image'].values:
        counter = 0
        
        img = load_img(i)
        normalized_img_array = img_to_array(img)

        start_h = used_image_size[0]
        end_h = used_image_size[1]
        normalized_img_array = normalized_img_array[start_h:end_h,:,:]

        ss.setBaseImage(normalized_img_array)
        ss.switchToSelectiveSearchFast()
        rects = ss.process()

        for (x, y, w, h) in rects:
            filename_box = filename
            proposed_box = [x, y, x + w, y + h]
            
            for x_d in bb_data.items():
                filename_bb = x_d[0]
                bounding_boxes = x_d[1]
                if filename_box == filename_bb:
                    image_array = normalized_img_array[proposed_box[1]:proposed_box[3],proposed_box[0]:proposed_box[2]]
                    image_array = cv2.resize(image_array,(target_w,target_h))

                    ious = []
                    for bounding_box in bounding_boxes:
                        xmin, ymin, xmax, ymax = bounding_box
                        ymin = ymin - start_h
                        ymax = ymax - start_h
                        bounding_box = [xmin, ymin, xmax, ymax]
                        ious.append(get_iou(bounding_box,proposed_box))
                    
                    if any(x > 0.6 for x in ious):
                        label = 1
                        pos_counter += 1
                        path = '../../data/training_images_labeled-3/for_rcnn/1/' + str(pos_counter) + '.jpg'
                        cv2.imwrite(path, image_array)
                    else:
                        label = 0
                        if (neg_counter - pos_counter) <= 10:
                            neg_counter += 1
                            path = '../../data/training_images_labeled-3/for_rcnn/0/' + str(neg_counter) + '.jpg'
                            cv2.imwrite(path, image_array)
                    
            counter += 1
            if counter >= max_proposed_boxes:
                break